In [52]:
import pygame
import numpy as np
import time
import random
import time

In [53]:
def coordinates_to_lists(coordinates):
    x_coordinates = []
    y_coordinates = []
    for i, value in enumerate(coordinates):
        if value == 1:
            x_coordinates.append(i % 4)
            y_coordinates.append(i // 4)
    return x_coordinates, y_coordinates

def update(cells):
    updated_cells = np.zeros((cells.shape[0],cells.shape[1]))
    for row, col in np.ndindex(cells.shape):
        alive = np.sum(cells[row-1:row+2, col-1:col+2]) - cells[row,col]

        #GAME RULES
        if cells[row,col] == 1:
            if 2 <= alive <= 3:
                updated_cells[row, col] = 1
        else:
            if alive ==3:
                updated_cells[row, col] = 1
    return(updated_cells)

def simulation(coordinates,toprint=False):
    x_coords, y_coords = coordinates_to_lists(coordinates)
    cells = np.zeros((60,80))

    cole = []
    rowe = []
    for i in range(len(x_coords)):
        cole.append(x_coords[i]+25)
        rowe.append(y_coords[i]+33)

    for i in range(len(cole)):
        cells[cole[i],rowe[i]] = 1

    most_cells = np.sum(cells)

    record_list = []
    for i in range(1000):
        amount_of_cells = np.sum(cells)
        if amount_of_cells == 0:
            break
        if i%100==0:
            if toprint:
                print(f"Cell count at {i} steps: {amount_of_cells}")
            record_list.append(amount_of_cells)
            if record_list.count(amount_of_cells) >2:
                if toprint:
                    print("-"*30)
                return(most_cells)
        cells = update(cells)
        if amount_of_cells > most_cells:
            most_cells = amount_of_cells

    #print(f"Highest cell count in 10,000 steps: {most_cells}")
    if toprint:
        print("-"*30)
    return(most_cells)

In [55]:
GRID_SIZE = 4
POPULATION_SIZE = 7
MUTATION_RATE = 0.1
GENERATIONS = 10
def generate_pattern():
    return [random.randint(0, 1) for _ in range(GRID_SIZE * GRID_SIZE)]

def initialize_population():
    return [generate_pattern() for _ in range(POPULATION_SIZE)]

def crossover(parent1, parent2):
    crossover_point = random.randint(1, GRID_SIZE * GRID_SIZE - 2)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(pattern):
    mutated_pattern = pattern[:]
    for i in range(len(mutated_pattern)):
        if random.random() < MUTATION_RATE:
            mutated_pattern[i] = 1 if mutated_pattern[i] == 0 else 0
    return mutated_pattern

def evolve_population(population):
    new_population = []
    scores = [simulation(pattern) for pattern in population]
    total_score = sum(scores)
    probabilities = [score / total_score for score in scores]
    
    for _ in range(POPULATION_SIZE // 2):
        parent1, parent2 = random.choices(population, weights=probabilities, k=2)
        child1, child2 = crossover(parent1, parent2)
        new_population.extend([mutate(child1), mutate(child2)])
    
    return new_population

population = initialize_population()
record_patterns = []
best_score_max = 0
best_pattern_max = []

for generation in range(GENERATIONS):
    print("Evolving population...")
    start_evolve = time.time()
    population = evolve_population(population)
    end_evolve = time.time()
    evolve_time = end_evolve - start_evolve
    print(f"Time to evolve: {evolve_time:.4f} seconds")

    start_score = time.time()
    print("Scoring new population")
    best_pattern = max(population, key=lambda pattern: simulation(pattern))
    best_score = simulation(best_pattern,toprint=True)
    end_score = time.time()
    score_time = end_score - start_score
    print(f"Time to Score: {score_time:.4f} seconds")
    print()
    record_patterns.append(best_pattern)
    if best_score > best_score_max:
        best_score_max = best_score
        best_pattern_max = best_pattern

    if record_patterns.count(best_pattern) > 2:
        print(f"Repeated {best_pattern} too many times")
        break
    print(f"Generation {generation + 1}, Pattern: {best_pattern}, Best Score: {best_score},\nProgress: {((generation + 1) / GENERATIONS) * 100}%")
print("Best Absolute pattern",best_pattern_max," Score:",best_score_max)

Evolving population...
Time to evolve: 30.8876 seconds
Scoring new population
Cell count at 0 steps: 8.0
Cell count at 100 steps: 79.0
Cell count at 200 steps: 112.0
Cell count at 300 steps: 148.0
Cell count at 400 steps: 114.0
Cell count at 500 steps: 108.0
Cell count at 600 steps: 76.0
Cell count at 700 steps: 84.0
Cell count at 800 steps: 72.0
Cell count at 900 steps: 76.0
------------------------------
Time to Score: 66.2060 seconds

Generation 1, Pattern: [0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1], Best Score: 239.0,
Progress: 10.0%
Evolving population...
Time to evolve: 45.7093 seconds
Scoring new population
Cell count at 0 steps: 9.0
------------------------------
Time to Score: 14.5231 seconds

Generation 2, Pattern: [0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0], Best Score: 43.0,
Progress: 20.0%
Evolving population...
Time to evolve: 13.6004 seconds
Scoring new population
Cell count at 0 steps: 10.0
Cell count at 100 steps: 17.0
Cell count at 200 steps: 17.0
Cell coun